In [184]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [185]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [186]:
spectrum_train_scaled = spectrum_train.multiply(10**12)
spectrum_test_scaled = spectrum_test.multiply(10**12)

In [187]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [188]:
input_data = df_to_tensor(temp_train)
output_data = df_to_tensor(spectrum_train_scaled)

In [189]:
input_data

tensor([[350.6424, 357.4240, 338.0000,  ..., 267.9975, 252.9999, 238.0000],
        [352.1875, 360.3629, 342.0451,  ..., 269.5426, 252.9999, 236.4549],
        [350.6424, 357.4240, 338.0000,  ..., 267.9975, 252.9999, 238.0000],
        ...,
        [597.3622, 585.6775, 574.4026,  ..., 493.7609, 505.0429, 515.4084],
        [595.8171, 582.7385, 570.3576,  ..., 492.2158, 505.0429, 516.9536],
        [601.9975, 594.4943, 586.5379,  ..., 498.3961, 505.0429, 510.7732]])

In [190]:
output_data

tensor([[ 0.1119,  0.0821,  0.0643,  ...,  0.3437,  0.3360,  0.3290],
        [ 0.1240,  0.0904,  0.0701,  ...,  0.3269,  0.3197,  0.3131],
        [ 0.1119,  0.0821,  0.0643,  ...,  0.3437,  0.3360,  0.3290],
        ...,
        [10.8134, 10.6301, 10.5865,  ..., 21.8142, 20.7956, 19.8604],
        [10.6221, 10.5147, 10.5277,  ..., 22.0530, 21.0220, 20.0754],
        [11.4288, 11.0081, 10.7881,  ..., 21.1053, 20.1236, 19.2221]])

In [191]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [192]:
input_size = 11
output_size = 66
model = Net(input_size=input_size, output_size=output_size)

In [193]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam optimizer

In [194]:
num_epochs = 100
batch_size = 32
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 10/100, Loss: 6.383619
Epoch 20/100, Loss: 8.144225
Epoch 30/100, Loss: 11.092988
Epoch 40/100, Loss: 19.777275
Epoch 50/100, Loss: 22.170179
Epoch 60/100, Loss: 21.935501
Epoch 70/100, Loss: 21.264130
Epoch 80/100, Loss: 20.477892
Epoch 90/100, Loss: 20.263536
Epoch 100/100, Loss: 20.305302


In [195]:
predictions = model(df_to_tensor(temp_train))

In [196]:
pd.DataFrame(predictions.detach().numpy(), columns=spectrum_test.columns).head()

,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,0.957020,0.823448,0.706688,0.745883,0.885037,0.672862,0.815175,0.802330,0.905220,0.869727,...,1.945669,1.855932,1.922320,1.913216,2.048449,1.995728,2.076719,2.051072,1.843851,1.887731
1,0.840085,0.644758,0.701988,0.644552,0.779771,0.532637,0.597444,0.697043,0.792870,0.687446,...,1.541514,1.558240,1.475706,1.540913,1.641199,1.627997,1.688710,1.685466,1.564429,1.575002
2,0.957020,0.823448,0.706688,0.745883,0.885037,0.672862,0.815175,0.802330,0.905220,0.869727,...,1.945669,1.855932,1.922320,1.913216,2.048449,1.995728,2.076719,2.051072,1.843851,1.887731
3,0.528585,0.407341,0.527655,0.439027,0.537667,0.188951,0.159860,0.601456,0.512289,0.221047,...,0.830938,1.005566,0.363318,0.779375,0.838134,0.909691,1.024762,0.903071,1.046144,0.868963
4,1.046932,0.922871,0.721506,0.824719,0.921470,0.774855,0.891189,0.903957,0.957144,0.990339,...,2.221480,2.061422,2.189156,2.104810,2.271966,2.216904,2.282218,2.255555,2.026062,2.036279


In [197]:
spectrum_test_scaled.head()

,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,2.373538,2.602481,2.775814,2.935397,3.074856,3.220465,3.363583,3.505908,3.647634,3.790090,...,10.069020,10.146292,10.208057,10.243674,10.246583,10.192531,10.036903,9.737349,9.326093,8.947765
1,2.027413,2.150030,2.250785,2.362463,2.467873,2.581041,2.694600,2.809533,2.925638,3.043688,...,8.794491,8.871859,8.934365,8.973221,8.983096,8.942860,8.813263,8.556884,8.201778,7.875021
2,2.470590,2.726463,2.921211,3.095970,3.246416,3.402639,3.555543,3.706972,3.857192,4.007671,...,10.460966,10.538142,10.599540,10.633985,10.634567,10.576088,10.412293,10.099323,9.670691,9.276407
3,1.950271,2.046041,2.131383,2.233903,2.333012,2.440313,2.548641,2.658761,2.770347,2.884035,...,8.542764,8.620148,8.682729,8.722118,8.733251,8.695631,8.571070,8.323129,7.979041,7.662406
4,2.523524,2.771409,2.957469,3.126776,3.273742,3.426844,3.576945,3.725913,3.874010,4.022686,...,10.456130,10.533535,10.595041,10.629530,10.630143,10.571716,10.408014,10.095198,9.666764,9.272662
